In [24]:
import jupyter
from tweet_browser_test import tweet_browser as tb
import ipywidgets as widgets
import voila
from matplotlib import pyplot as plt
from ipysheet import sheet, cell, row, column, cell_range
from IPython.display import display


colHeaders = ["SenderScreenName", "CreatedTime", "Message"]   
operators = [None] * 5
#currentOperator = 0
s = tb.createSession("allCensus_sample.csv")
#output = []
out = widgets.Output()

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[tb.headerDict[j]])
    return result

def resetDisplay():
    global operator
    out.clear_output()
    with out:
        display(grouped_buttons)
        display(operator)
def clear(b):
    resetDisplay()

def randomSample(b):
    if s.currentSet != s.base:
        s.back()
    s.simpleRandomSample(sampleSelector.value)
    getTweets(b)

def weightedSample(b):
    if s.currentSet != s.base:
        s.back()
    s.weightedSample(sampleSelector.value, weightedSelector.value)
    getTweets(b)
    
def searchKeyword(b):
    if s.currentSet != s.base:
        s.back()
    if searchType.value == 'Contains Any':
        s.searchKeyword(searchBox.value.split(' '), True)
    elif searchType.value == 'Contains All':
        s.searchKeyword(searchBox.value.split(' '))
    elif searchType.value == 'Advanced':
        s.advancedSearch(searchBox.value)
    elif searchType.value == 'Regular Expression':
        s.regexSearch(searchBox.value)
    else:
        return
    getTweets(b)
    
def filterBy(b):
    if s.currentSet != s.base:
        s.back()
    index = filterBox.value.find(" ")
    if index == -1:
        print("Invalid input")
        return
    columnName = filterBox.value[:index]
    if not (columnName in tb.headerDict):
        print("Invalid column")
        return
    value = filterBox.value[index + 1:]
    s.filterBy(columnName, value)
    getTweets(b)

def dimensionHandler(change):
    global operators, operator, clusterBar, clusteringWhen
    if change['new'] > 2:
        clusteringWhen = widgets.ToggleButtons(options=[("Before stage 1", "before_stage1"), 
                                ("Between 1 and 2", "btwn"),("After stage 2", "after_stage2")], 
                                value= "before_stage1", description= "When to cluster: ")
        clusterBar = clusterBar = widgets.VBox([min_df, dimRed1, dimensions, dimRed2, clusteringWhen, 
                                                clusteringMethod, clusteringInput, clusterButton])

    else:
        clusteringWhen = widgets.ToggleButtons(options=[("Before stage 1", "before_stage1"), ("Between 1 and 2", "btwn")], 
                                value= "before_stage1", description= "When to cluster: ")
        clusterBar = widgets.VBox([min_df, dimRed1, dimensions, clusteringWhen, 
                                   clusteringMethod, clusteringInput, clusterButton])
    operators[4] = clusterBar
    operator = operators[4]
    resetDisplay()

def cluster(b):
    #s.filterBy("State", "New Jersey")
    #s.regexSearch("Trump")
    matrix, words = s.make_full_docWordMatrix(min_df= min_df.value)
    result = s.dimRed_and_clustering(docWordMatrix_orig=matrix, dimRed1_method=dimRed1.value, 
                                     dimRed1_dims=dimensions.value, dimRed2_method=dimRed2.value, 
                                     clustering_when=clusteringWhen.value, clustering_method1=clusteringMethod.value, 
                                     num_clusters=clusteringInput.value, min_obs=clusteringInput.value, 
                                     num_neighbors=clusteringInput.value)
    with out:
        result.show()
    return result
    

def getTweets(b):
    resetDisplay()
    htmlText = "<style>table, th, td {border: 1px solid black;} th, td {padding: 10px;}</style>"
    htmlText += "<table> "
    dataSet = s.getCurrentSubset()
    for i in dataSet:
        htmlText += " <tr> "
        cols = selectColumns(i, colHeaders)
        for j in cols:
            htmlText += " <td>" + j + "</td> "
        htmlText += " </tr> "
    table = widgets.HTML(value=htmlText + " </table>")
    with out:
        print("Tweets displayed:", s.currentSet.size)
        display(table)

####### UI Widgets #######

# Selection Dropdown
selector = widgets.Dropdown(
    options=['Random Sample', 'Weighted Sample', 'Search Keyword', 'Filter By', 'Clustering'],
    value='Random Sample', description='Operation:')
clearButton = widgets.Button(description='Clear Tweets', tooltip='Clear current Tweets')
grouped_buttons = widgets.HBox([selector, clearButton])
clearButton.on_click(clear)

# Random Sample
randomSampleButton = widgets.Button(description='Get Tweets', tooltip='Get Tweets')   
sampleSelector = widgets.IntSlider(value=300, min=0, max=s.currentSet.size, step=1, description='Sample size',
    continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
randomSampleBar = widgets.HBox([sampleSelector, randomSampleButton])
randomSampleButton.on_click(randomSample)
operators[0] = randomSampleBar

# Weighted Sample
weightedButton = widgets.Button(description='Get Tweets', tooltip='Get Tweets')  
weightedSelector = widgets.Text(value='', placeholder='<Column_name>', description='Weight by')
weightedBar = widgets.HBox([sampleSelector, weightedSelector, weightedButton])
weightedButton.on_click(weightedSample)
operators[1] = weightedBar

#Search Bar
searchBox = widgets.Text(value='', placeholder='Keyword', description='Input:')
searchButton = widgets.Button(description='Search', tooltip='Run search on current input')
ContainsAnyToolTip = 'Returns tweets with any typed keyword (seperated by spaces)'
ContainsAllToolTip = 'Returns tweets containing all typed space-separated keywords'
AdvancedSearchToolTip = 'Search for tweets that match the inputted logical expression'
RegularExpressionToolTip = 'Returns tweets that match regular expression'
searchType = widgets.ToggleButtons(options=['Contains Any', 'Contains All', 'Advanced', 'Regular Expression'],
    tooltips=[ContainsAnyToolTip, ContainsAllToolTip, AdvancedSearchToolTip, RegularExpressionToolTip])
searchBar = widgets.VBox([widgets.HBox([searchBox, searchButton]), searchType])
searchButton.on_click(searchKeyword)
operators[2] = searchBar

#Filer By
filterBox = widgets.Text(value='', placeholder='<Column_name> <value>', description='Filter:')
filterButton = widgets.Button(description='Filter', tooltip='Filter by column on current input')
filterBar = widgets.HBox([filterBox, filterButton])
filterButton.on_click(filterBy)
operators[3] = filterBar

#Clustering
min_df = widgets.IntText(value= 1, description= "Word Frequency", style={'description_width': 'initial'},
                          tooltip= "Specifies the minum frequency required for a word to be considered during processing")
dimRed1 = widgets.ToggleButtons(options=[('PCA', "pca"), ("UMAP", "umap")], value="pca", description="First dimension reduction method: ")
dimensions = widgets.IntText(value= 2, description= "Dimensions: " )
dimRed2 = widgets.ToggleButtons(options=[('PCA', "pca"), ("UMAP", "umap")], value="pca", description="Second dimension reduction method: ")
clusteringWhen = widgets.ToggleButtons(options=[("Before stage 1", "before_stage1"), ("Between 1 and 2", "btwn")], 
                                      value= "before_stage1", description= "When to cluster: ")
clusteringMethod = widgets.ToggleButtons(options=[("GMM", "gmm"), ("K-Means", "k-means"), ("HDBScan", "hdbscan"), ("Leiden", "leiden")], 
                                         value="gmm", description="Clustering method ")
clusteringInput = widgets.IntText(description= "Number of Clusters: ", value= 2, style={'description_width': 'initial'})
clusterButton = widgets.Button(description='Cluster', tooltip= 'Start clustering')
clusterBar = widgets.VBox([min_df, dimRed1, dimensions, clusteringWhen, clusteringMethod, clusteringInput, clusterButton])
clusterButton.on_click(cluster)
dimensions.observe(dimensionHandler, names='value')
operators[4] = clusterBar

    
operator = operators[0]
with out:
    display(grouped_buttons)
    display(operator)
#operator.layout.visibility = 'hidden'

def selctionHandler(change):
    global operator
    operator = operators[change['owner'].index]
    resetDisplay()
    
selector.observe(selctionHandler, names='value')
out

HTML(value='Hello <b>World</b>')

Output()